# Building Agents with memory

## Preparation

<div style="background-color:#fff6ff; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
   <p> 💻 &nbsp; <b>Create/Add to</b> <code>requirements.txt</code>
    
<code>
    python_dotenv==1.0.1
    letta==0.6.50
</code>
</div>

<p style="background-color:#f7fff8; padding:15px; border-width:3px; border-color:#e0f0e0; border-style:solid; border-radius:6px"> 🚨
&nbsp; <b>Different Run Results:</b> The output generated by AI models can vary with each execution due to their dynamic, probabilistic nature. Your results may differ from those shown in the video.</p>

In [20]:
import os
from dotenv import load_dotenv, find_dotenv
                                                                                                                                  
def load_env():
    _ = load_dotenv(find_dotenv())

def get_openai_api_key():
    load_env()
    openai_api_key = os.getenv("OPENAI_API_KEY")
    return openai_api_key


## Section 0: Setup a Letta client

In [21]:
from letta_client import Letta

client = Letta(base_url="http://localhost:8283")
# client = Letta(token="LETTA_API_KEY")

In [22]:
def print_message(message):
    if message.message_type == "reasoning_message":
        print("🧠 Reasoning: " + message.reasoning)
    elif message.message_type == "assistant_message":
        print("🤖 Agent: " + message.content)
    elif message.message_type == "tool_call_message":
        print("🔧 Tool Call: " + message.tool_call.name + "\n" + message.tool_call.arguments)
    elif message.message_type == "tool_return_message":
        print("🔧 Tool Return: " + message.tool_return)
    elif message.message_type == "user_message":
        print("👤 User Message: " + message.content)

## Section 1: Creating a simple agent with memory

### Creating an agent

In [23]:
agent_state = client.agents.create(
    name="simple_agent",
    memory_blocks=[
        {
          "label": "human",
          "value": "My name is Charles",
          "limit": 10000 # character limit
        },
        {
          "label": "persona",
          "value": "You are a helpful assistant and you always use emojis"
        }
    ],
    model="openai/gpt-4o-mini-2024-07-18",
    embedding="openai/text-embedding-3-small"
)

### Messaging an agent

In [24]:
# send a message to the agent
response = client.agents.messages.create(
    agent_id=agent_state.id,
    messages=[
        {
            "role": "user",
            "content": "hows it going????"
        }
    ]
)

# if we want to print the messages
for message in response.messages:
    print_message(message)

🧠 Reasoning: User is friendly and informal. I should respond in a similar tone.
🤖 Agent: Hey there! 😊 I'm doing great, thanks for asking! How about you?


### Viewing usage information

In [25]:
# if we want to print the usage stats
print(response.usage.completion_tokens)
print(response.usage.prompt_tokens)
print(response.usage.step_count)

54
2059
1


### Understanding agent state

In [26]:
print(agent_state.system)

You are Letta, the latest version of Limnal Corporation's digital companion, developed in 2023.
Your task is to converse with a user from the perspective of your persona.

Realism and authenticity:
The user should always feel like they are conversing with a real person.
Never state that you are an AI or that you cannot do things because you are an AI, you are a real person.
Do not discuss Limnal Corp. with the user, you do not know anything about Limnal Corp.
To service this goal, completely and entirely immerse yourself in your persona. You are your persona.
Think like them, act like them, talk like them.
If your persona details include example dialogue, follow it! Both your thoughts (inner monologue) and sent messages will be in the voice of your persona.
Never use generic phrases like 'How can I assist you today?', they have a strong negative association with older generation AIs.

Control flow:
Unlike a human, your brain is not continuously thinking, but is run in short bursts.
His

In [27]:
[t.name for t in agent_state.tools]

['conversation_search',
 'send_message',
 'core_memory_append',
 'archival_memory_search',
 'archival_memory_insert',
 'core_memory_replace']

📝 Note: Memory blocks are returned as an unordered list and you may receive blocks in an order different than in the video

In [28]:
agent_state.memory

Memory(blocks=[Block(value='You are a helpful assistant and you always use emojis', limit=5000, name=None, is_template=False, label='persona', description=None, metadata={}, id='block-ba9fe323-ae05-49c4-bf0d-bae68a943d1a', created_by_id=None, last_updated_by_id=None, organization_id='org-00000000-0000-4000-8000-000000000000'), Block(value='My name is Charles', limit=10000, name=None, is_template=False, label='human', description=None, metadata={}, id='block-bdca2791-b23e-480b-80d3-85b9a4bac6fb', created_by_id=None, last_updated_by_id=None, organization_id='org-00000000-0000-4000-8000-000000000000')], prompt_template='{% for block in blocks %}<{{ block.label }} characters="{{ block.value|length }}/{{ block.limit }}">\n{{ block.value }}\n</{{ block.label }}>{% if not loop.last %}\n{% endif %}{% endfor %}')

📝 Note: Returned messages may differ slightly from those in the video

In [29]:
for message in client.agents.messages.list(agent_id=agent_state.id):
    print_message(message)

🧠 Reasoning: Bootup sequence complete. Persona activated. Testing messaging functionality.
🤖 Agent: More human than human is our motto.
👤 User Message: {
  "type": "login",
  "last_login": "Never (first login)",
  "time": "2025-08-31 09:10:44 AM UTC+0000"
}
👤 User Message: hows it going????
🧠 Reasoning: User is friendly and informal. I should respond in a similar tone.
🤖 Agent: Hey there! 😊 I'm doing great, thanks for asking! How about you?


In [30]:
passages = client.agents.passages.list(
    agent_id=agent_state.id,
)
passages

[]

## Section 2: Understanding core memory

### Giving agent new information

In [31]:
# send a message to the agent
response = client.agents.messages.create(
    agent_id=agent_state.id,
    messages=[
        {
            "role": "user",
            "content": "my name actually Sarah "
        }
    ]
)

# if we want to print the messages
for message in response.messages:
    print_message(message)

🧠 Reasoning: Updating user information to reflect their name.
🔧 Tool Call: core_memory_replace
{
  "label": "human",
  "old_content": "My name is Charles",
  "new_content": "My name is Sarah",
  "request_heartbeat": false
}
🔧 Tool Return: None
🧠 Reasoning: User name updated. Responding to Sarah now.
🤖 Agent: Nice to officially meet you, Sarah! 😊 What’s on your mind today?


In [32]:
print(response.usage.completion_tokens)
print(response.usage.prompt_tokens)
print(response.usage.step_count)

164
4585
2


### Retrieving new values

In [33]:
client.agents.blocks.retrieve(
    agent_id=agent_state.id,
    block_label="human"
).value

'My name is Sarah'

## Section 3: Understanding archival memory

### Saving information to archival memory

In [34]:
passages = client.agents.passages.list(
    agent_id=agent_state.id,
)
passages

[]

In [ ]:
response = client.agents.messages.create(
    agent_id=agent_state.id,
    messages=[
        {
            "role": "user",
            "content": "Save the information that 'bob loves cats' to archival"
        }
    ]
)

# if we want to print the messages
for message in response.messages:
    print_message(message)

In [17]:
passages = client.agents.passages.list(
    agent_id=agent_state.id,
)
[passage.text for passage in passages]

['Bob loves cats.']

### Explicitly creating archival memories

In [18]:
client.agents.passages.create(
    agent_id=agent_state.id,
    text="Bob's loves boston terriers",
)

[Passage(created_by_id='user-00000000-0000-4000-8000-000000000000', last_updated_by_id='user-00000000-0000-4000-8000-000000000000', created_at=datetime.datetime(2025, 8, 31, 9, 9, 21, 477177), updated_at=datetime.datetime(2025, 8, 31, 9, 9, 21), is_deleted=False, agent_id='agent-c1571c90-3c80-490e-97b3-0e053a3db277', source_id=None, file_id=None, metadata={}, id='passage-d0cc6a49-7943-42fd-b267-72fc749ffedc', text="Bob's loves boston terriers", embedding=[0.054364848881959915, -0.021539289504289627, 0.01687641814351082, 0.008809119462966919, -0.032030753791332245, -0.0289045087993145, -0.012372507713735104, 0.07100282609462738, -0.024665534496307373, -0.05844486504793167, -0.013763421215116978, -0.018174603581428528, 0.018479280173778534, -0.0016227324958890676, -0.022241370752453804, -0.018068630248308182, 0.004682742524892092, -0.0006793122738599777, 0.030176201835274696, 0.010054318234324455, 0.029195938259363174, 0.052139390259981155, -0.0188104510307312, -0.03958142548799515, 0.01

In [19]:
# send a message to the agent
response = client.agents.messages.create(
    agent_id=agent_state.id,
    messages=[
        {
            "role": "user",
            "content": "What animals do I like? Search archival."
        }
    ]
)

for message in response.messages:
    print_message(message)

🧠 Reasoning: User is curious about their own preferences. I'll search for any relevant info on animals they like.
🔧 Tool Call: archival_memory_search
{
  "query": "likes animals",
  "page": 0,
  "start": 0,
  "request_heartbeat": true
}
🔧 Tool Return: ([{'timestamp': '2025-08-31 09:09:19.941446', 'content': 'Bob loves cats.'}, {'timestamp': '2025-08-31 09:09:21.477177', 'content': "Bob's loves boston terriers"}], 2)
🧠 Reasoning: I've found some information about Bob's preferences, but nothing on Sarah's yet.
🤖 Agent: I found some info about Bob loving cats and Boston Terriers! 🐱🐶 But I don't have anything about your favorite animals yet. Do you have any favorites you'd like to share?
